In [ ]:
import numpy as np
import networkx as nx
import pickle
import community
from operator import itemgetter
from scipy import integrate
from matplotlib import pyplot as plt
%matplotlib inline
import os
import pandas as pd
import sys

## [abo | gun | blm]
campaign = 'blm'
## [followers | friends]
connection_type = 'followers'
## [2017 | 2018 | 2020]
year = 2018
filter_type = 'disparity'


''' 
M. A. Serrano et al. (2009) Extracting the Multiscale backbone of complex weighted networks. PNAS, 106:16, pp. 6483-6488.
'''
def disparity_filter(G, weight='weight'):
    if nx.is_directed(G): #directed case    
        N = nx.DiGraph()
        for u in G:
            
            k_out = G.out_degree(u)
            k_in = G.in_degree(u)
            
            if k_out > 1:
                sum_w_out = sum(np.absolute(G[u][v][weight]) for v in G.successors(u))
                for v in G.successors(u):
                    w = G[u][v][weight]
                    p_ij_out = float(np.absolute(w))/sum_w_out
                    alpha_ij_out = 1 - (k_out-1) * integrate.quad(lambda x: (1-x)**(k_out-2), 0, p_ij_out)[0]
                    N.add_edge(u, v, weight = w, alpha_out=float('%.4f' % alpha_ij_out))
                    
            elif k_out == 1 and G.in_degree(G.successors(u)[0]) == 1:
                #we need to keep the connection as it is the only way to maintain the connectivity of the network
                v = G.successors(u)[0]
                w = G[u][v][weight]
                N.add_edge(u, v, weight = w, alpha_out=0., alpha_in=0.)
                #there is no need to do the same for the k_in, since the link is built already from the tail
            
            if k_in > 1:
                sum_w_in = sum(np.absolute(G[v][u][weight]) for v in G.predecessors(u))
                for v in G.predecessors(u):
                    w = G[v][u][weight]
                    p_ij_in = float(np.absolute(w))/sum_w_in
                    alpha_ij_in = 1 - (k_in-1) * integrate.quad(lambda x: (1-x)**(k_in-2), 0, p_ij_in)[0]
                    N.add_edge(v, u, weight = w, alpha_in=float('%.4f' % alpha_ij_in))
        return N
    
    else: #undirected case
        '''
        B = nx.Graph()
        for u in G:
            k = len(G[u])
            if k > 1:
                sum_w = sum(np.absolute(G[u][v][weight]) for v in G[u])
                for v in G[u]:
                    w = G[u][v][weight]
                    p_ij = float(np.absolute(w))/sum_w
                    alpha_ij = 1 - (k-1) * integrate.quad(lambda x: (1-x)**(k-2), 0, p_ij)[0]
                    B.add_edge(u, v, weight = w, alpha=float('%.4f' % alpha_ij))
        return B
        '''
        aa = 0
        for u in G:
            k = len(G[u])
            if k > 1:
                sum_w = sum(np.absolute(G[u][v][weight]) for v in G[u])
                for v in G[u]:
                    w = G[u][v][weight]
                    p_ij = float(np.absolute(w))/sum_w
                    alpha_ij = 1 - (k-1) * integrate.quad(lambda x: (1-x)**(k-2), 0, p_ij)[0]
                    #G.edges[v, w]['community'] = G.nodes[v]['community']
                    G[u][v]['alpha'] = float('%.4f' % alpha_ij)
            if aa % 1000 == 0:
                print(aa)
            aa+=1
        return G

''' 
M. A. Serrano et al. (2009) Extracting the Multiscale backbone of complex weighted networks. PNAS, 106:16, pp. 6483-6488.
'''
def disparity_filter_alpha_cut(G,weight='weight',alpha_t=0.4, cut_mode='or'):    
    if nx.is_directed(G):#Directed case:   
        B = nx.DiGraph()
        for u, v, w in G.edges(data=True):
            try:
                alpha_in =  w['alpha_in']
            except KeyError: #there is no alpha_in, so we assign 1. It will never pass the cut
                alpha_in = 1
            try:
                alpha_out =  w['alpha_out']
            except KeyError: #there is no alpha_out, so we assign 1. It will never pass the cut
                alpha_out = 1  
            
            if cut_mode == 'or':
                if alpha_in<alpha_t or alpha_out<alpha_t:
                    B.add_edge(u,v, weight=w[weight])
            elif cut_mode == 'and':
                if alpha_in<alpha_t and alpha_out<alpha_t:
                    B.add_edge(u,v, weight=w[weight])
        return B

    else:
        B = nx.Graph()#Undirected case:   
        for u, v, w in G.edges(data=True):
            
            try:
                alpha = w['alpha']
            except KeyError: #there is no alpha, so we assign 1. It will never pass the cut
                alpha = 1
                
            if alpha<alpha_t:
                B.add_edge(u,v, weight=w[weight])
        return B                


## Apply disparity filtering to shared audience graph of early adopters.

In [ ]:
## Read the connection list
connection_list = pickle.load(open("data/social_media/{}/ea_{}_{}.pkl".format(campaign, connection_type, str(year)), 'rb'))
print("{} list read!".format(connection_type))

## Read the connection graph
G = None
G2 = None
already_filtered = False
filtered_graph_file_path = 'data/social_media/{}/graph_edges/ea_filtered_graph_edge_list_{}.gpickle'.format(campaign, str(year))
if os.path.exists(filtered_graph_file_path):
    G = nx.read_gpickle(filtered_graph_file_path)
    already_filtered = True
    print('Filtered G read!')
else:
    complete_graph_file_path = 'data/social_media/{}/graph_edges/ea_{}_{}.txt'.format(campaign, connection_type, str(year))
    G = nx.read_weighted_edgelist(complete_graph_file_path)
    print("Initial graph G read!")

print("# of nodes in G:", G.number_of_nodes())
print("# of edges in G:", G.number_of_edges())


## Add the nodes not having any common followers to the graph.
diff_users = list(set(connection_list.keys()).difference(set(G.nodes)))
print("# of diff users:", len(diff_users))
for user in diff_users:
    G.add_node(user)

print("# of nodes in G after addition:", G.number_of_nodes())
print("# of edges in G after addition:", G.number_of_edges())


## Apply filtering to G if not not already filtered
if already_filtered == False:
    alpha = 0.05
    G = disparity_filter(G)
    G2 = nx.Graph([(u, v, d) for u, v, d in G.edges(data=True) if d['alpha'] < alpha and d['weight'] > 0])
    
    print("# of nodes in G2 after filtering:", G2.number_of_nodes())
    print("# of edges in G2 after filtering:", G2.number_of_edges())
    
    ## save the filtered network.
    nx.write_gpickle(G2, filtered_graph_file_path)
    print(">>> Disparity filtering finished! >>>")
    
else:
    print(">>> Filtered graph already exists! >>>")

